# ChAII - Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
# !pip uninstall fsspec -qq -y
!pip install datasets

     |████████████████████████████████| 264 kB 593 kB/s 
     |████████████████████████████████| 243 kB 7.6 MB/s 


In [2]:
sub = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
sub.head(1)

,id,PredictionString
0,22bff3dec,NaN


### What train data do we have available? 

Looks like the training set is very small: 747 Hindi and 368 Tamil examples. We have the context, question, answer_text and answer_start. So this is in fact a **reading comprehension** task, which means that we have context available and need to find the start and end span in that contexts that answers our question. Typical example of such format is SQUAD. 

In [3]:
train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
train.head()

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil


### What about test? 

In test, we have just 5 examples, and similar columns like in train except that we don't have *answer_text* and *answer_start*. It seems that *answer_text* is what we're trying to predict. 

In [4]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test.head()

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


# Baseline

Based on: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb

In [5]:
import transformers
from transformers import AutoTokenizer
from datasets import Dataset

model_checkpoint = '../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'
batch_size = 4

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pad_on_right = tokenizer.padding_side == "right"

max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed

In [6]:
def convert_answers(r):
    start = r[0]
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

In [7]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [8]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [9]:
train['num_tokens_context'] = train['context'].apply(lambda t: len(tokenizer(t)['input_ids']))
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1492 > 512). Running this sequence through the model will result in indexing errors


In [10]:
train = train.drop(['answer_start', 'answer_text'], axis=1)
train.head()

,id,context,question,language,num_tokens_context,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,tamil,1492,"{'answer_start': [53], 'text': ['206']}"
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,tamil,2316,"{'answer_start': [2358], 'text': ['காசுமீரில்']}"
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,tamil,1470,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட..."
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,tamil,2715,"{'answer_start': [68], 'text': ['தாலாட்டு']}"
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,tamil,2821,"{'answer_start': [585], 'text': ['சூரியனும்']}"


In [11]:
train_hindi = train.loc[train['language'] == 'hindi']
train_tamil = train.loc[train['language'] == 'tamil']

train_hindi = train_hindi.sample(frac=1, random_state=42)
train_tamil = train_tamil.sample(frac=1, random_state=42)

print(len(train_hindi), len(train_tamil))

746 368


## Hindi

In [12]:
df_train_hindi = train_hindi[:-64].reset_index(drop=True)
df_valid_hindi = train_hindi[-64:].reset_index(drop=True)

train_dataset_hindi = Dataset.from_pandas(df_train_hindi)
valid_dataset_hindi = Dataset.from_pandas(df_valid_hindi)

tokenized_train_ds_hindi = train_dataset_hindi.map(prepare_train_features, batched=True, remove_columns=train_dataset_hindi.column_names)
tokenized_valid_ds_hindi = valid_dataset_hindi.map(prepare_train_features, batched=True, remove_columns=train_dataset_hindi.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
train_dataset_hindi[0]

{'id': 'b5ef4590a',
 'context': 'बुर्ज ख़लीफ़ा दुबई में आठ अरब डॉलर की लागत से छह साल में निर्मित ८२८ मीटर ऊँची १६८ मंज़िला दुनिया की सबसे ऊँची इमारत है (जनवरी, सन् २०१० में)। इसका लोकार्पण ४ जनवरी, २०१० को भव्य उद्घाटन समारोह के साथ किया गया। इसमें तैराकी का स्थान, खरीदारी की व्यवस्था, दफ़्तर, सिनेमा घर सहित सारी सुविधाएँ मौजूद हैं। इसकी ७६ वीं मंजिल पर एक मस्जिद भी बनायी गयी है। इसे ९६ किलोमीटर दूर से भी साफ़-साफ़ देखा जा सकता है। इसमें लगायी गयी लिफ़्ट दुनिया की सबसे तेज़ चलने वाली लिफ़्ट है। “ऐट द टॉप” नामक एक दरवाज़े के बाहर अवलोकन डेक, 124 वीं मंजिल पर, 5 जनवरी 2010 पर खुला। यह 452 मीटर (1,483 फुट) पर, दुनिया में तीसरे सर्वोच्च अवलोकन डेक और दुनिया में दूसरा सबसे बड़ा दरवाज़े के बाहर अवलोकन डेक है।\nनिर्माण विशेषता सन्दर्भ\nबाहरी\xa0कड़ियाँ\nNo URL found. Please specify a URL here or add one to Wikidata.\nश्रेणी: गगनचुम्बी इमारतें\nश्रेणी: सर्वोच्च गगनचुम्बी',
 'question': 'बुर्ज खलीफा कहाँ स्थित है?',
 'language': 'hindi',
 'num_tokens_context': 258,
 'answers': {'answer_start':

## Tamil

In [14]:
df_train_tamil = train_tamil[:-64].reset_index(drop=True)
df_valid_tamil = train_tamil[-64:].reset_index(drop=True)

train_dataset_tamil = Dataset.from_pandas(df_train_tamil)
valid_dataset_tamil = Dataset.from_pandas(df_valid_tamil)

tokenized_train_ds_tamil = train_dataset_tamil.map(prepare_train_features, batched=True, remove_columns=train_dataset_tamil.column_names)
tokenized_valid_ds_tamil = valid_dataset_tamil.map(prepare_train_features, batched=True, remove_columns=train_dataset_tamil.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
train_dataset_tamil[0]

{'id': '83f5c039a',
 'context': 'மைக்ரோசாப்ட் நிறுவனம்(Microsoft Corporation) அமெரிக்காவை மையமாக கொண்டு செயல் படும் ஓர் பன்னாட்டு மென்பொருள் தொழில்நுட்ப நிறுவனம் ஆகும். இது உலகின் மிகப் பெரிய மென்பொருள் தயாரிப்பு நிறுவனமாக விளங்குகிறது. கணினிக்குத் தேவையான பல வகை மென்பொருட்களை தயாரிப்பது, மேம்படுத்துவது, உரிமை மற்றும் ஆதரவு அளிப்பது போன்ற செயற்பாடுகளை உடையது. வாசிங்டனில் உள்ள ரெட்மாண்ட் நகரத்தில் இதன் தலைமை இடம் உள்ளது. இதனை பில் கேட்சும் பவுல் ஆல்லெனும் ஏப்ரல் 4, 1975இல் நிறுவினர். வருமானத்தின் அளவுகொண்டு உலகின் மிகப்பெரும் மென்பொருள் ஆக்குனராக மைக்ரோசாப்ட் விளங்குகிறது.[4] உலகளவில் பங்குச்சந்தையில் மிகக் கூடுதலான மொத்தமதிப்பைக் கொண்டுள்ள நிறுவனமாகவும் இது விளங்குகிறது.[5]விண்டோஸ் இயக்குதளங்கள், மைக்ரோசாப்ட் ஆபிஸ் ஆகியவை இதன் முக்கிய உற்பத்தி பொருட்கள் ஆகும்.\nஅல்டைர் 8800விற்கு பேசிக் மொழிமாற்றி மென்பொருளை உருவாக்கி விற்கும் நிறுவனமாக மைக்ரோசாப்ட் முதலில் துவங்கப்பட்டது. பின்னர் எம்.எஸ்.டாஸ் எனும் இயக்குதளத்தை 1980களில் அறிமுகப்படுத்தி தனி மேசைக் கணினி இயக்கு தளம் தயாரிப்பில் முன்னணி

## Training

In [16]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

In [17]:
# import wandb
# wandb.login()
# %env WANDB_PROJECT=chaii
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


## Tamil Train

In [18]:
model_checkpoint = '../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
data_collator = default_data_collator

args = TrainingArguments(
    f"chaii-qa-2",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
#     report_to="wandb"
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds_tamil,
    eval_dataset=tokenized_valid_ds_tamil,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.train()
# wandb.finish()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
0,No log,0.322845


TrainOutput(global_step=144, training_loss=0.3162834644317627, metrics={'train_runtime': 741.3881, 'train_samples_per_second': 0.194, 'total_flos': 0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 399462400, 'init_mem_gpu_alloc_delta': 2235376640, 'init_mem_cpu_peaked_delta': 1594335232, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 90677248, 'train_mem_gpu_alloc_delta': 6733405184, 'train_mem_cpu_peaked_delta': 3076829184, 'train_mem_gpu_peaked_delta': 5504041984})

In [19]:
trainer.save_model("chaii-bert-trained-tamil")

## Hindi Train

In [20]:
model_checkpoint = '../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
data_collator = default_data_collator

In [21]:
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
#     report_to="wandb"
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds_hindi,
    eval_dataset=tokenized_valid_ds_hindi,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
# wandb.finish()

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
0,No log,0.203466


TrainOutput(global_step=262, training_loss=0.2588689381839665, metrics={'train_runtime': 1270.7007, 'train_samples_per_second': 0.206, 'total_flos': 0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': -1023795200, 'init_mem_gpu_alloc_delta': 2236417024, 'init_mem_cpu_peaked_delta': 1023795200, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 45056, 'train_mem_gpu_alloc_delta': 6729210880, 'train_mem_cpu_peaked_delta': 2047791104, 'train_mem_gpu_peaked_delta': 5503517696})

In [22]:
trainer.save_model("chaii-bert-trained-hindi")

## Validate

In [23]:
validation_features_hindi = valid_dataset_hindi.map(
    prepare_validation_features,
    batched=True,
    remove_columns=valid_dataset_hindi.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
valid_feats_small = validation_features_hindi.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

  0%|          | 0/843 [00:00<?, ?ex/s]

In [25]:
raw_predictions = trainer.predict(valid_feats_small)

In [26]:
import collections

max_answer_length = 30
examples = valid_dataset_hindi
features = validation_features_hindi

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [27]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [28]:
final_predictions = postprocess_qa_predictions(valid_dataset_hindi, validation_features_hindi, raw_predictions.predictions)

Post-processing 64 example predictions split into 843 features.


  0%|          | 0/64 [00:00<?, ?it/s]

In [29]:
references = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in valid_dataset_hindi]

In [30]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [31]:
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res

,id,answer,prediction,jaccard
0,151071dab,‘आर्थोपोडा,आर्थोपोडा,0.000000
1,4a92c37c2,1613 ई.,1613 ई.,1.000000
2,c20772e17,1927,1927,1.000000
3,455f23be7,एस एन डी टी,एस एन डी टी,1.000000
4,eddadac58,स्वामी हरिदास,स्वामी हरिदास जी,0.666667
...,...,...,...,...
59,58b3676a4,मराठी,मराठी,1.000000
60,5aeacd81a,सेंट्रल प्रोसेसिंग यूनिट,सेंट्रल प्रोसेसिंग यूनिट,1.000000
61,89d938493,इब्न-अल-हज़ैन,इब्न-अल-हज़ैन,1.000000
62,8d13dfd40,फ़्रांसिसी,फ़्रांसिसी,1.000000


In [32]:
res.jaccard.mean()

0.6985677083333334